# Base Theory for Angle of Arrival

For a ULA with N elements spaced by d, an incoming plane wave at an angle θ results in a phase difference between adjacent elements

# Uniform Linear Array (ULA) at 2.4 GHz

Assumptions:

    Frequency f=2.4f=2.4 GHz
    Wavelength λ=cfλ=fc​ where c=3×108c=3×108 m/s
    Element spacing d=0.5λd=0.5λ (to avoid grating lobes)
    N=8N=8 elements
    Incident wave at 30°

In [1]:
import numpy as np

In [2]:

# Constants
f = 2.4e9  # Frequency in Hz
c = 3e8  # Speed of light in m/s
lambda_ = c / f  # Wavelength in meters
d = 0.5 * lambda_  # Element spacing (half-wavelength)
N = 3  # Number of elements
theta = np.radians(30)  # Angle of Arrival in radians

# Phase shift per element
delta_phi = (2 * np.pi * d / lambda_) * np.sin(theta)

# Compute phase shift for each element
phase_shifts = np.arange(N) * delta_phi

# Print results
for i, phase in enumerate(phase_shifts):
    print(f"Element {i}: Phase shift = {phase:.4f} radians")


Element 0: Phase shift = 0.0000 radians
Element 1: Phase shift = 1.5708 radians
Element 2: Phase shift = 3.1416 radians


In [24]:
# Phase shift per element (convert to degrees)
delta_phi = np.degrees((2 * np.pi * d / lambda_) * np.sin(theta))

# Compute phase shift for each element
phase_shifts = np.arange(N) * delta_phi

# Print results
for i, phase in enumerate(phase_shifts):
    print(f"Element {i}: Phase shift = {phase:.2f} degrees")

Element 0: Phase shift = 0.00 degrees
Element 1: Phase shift = 90.00 degrees
Element 2: Phase shift = 180.00 degrees


## Angle of Arrival Estimation

In [4]:
def estimate_aoa(phase_measurements, f, d_factor=0.5):
    """
    Estimate AoA from three antenna phase measurements.
    
    Parameters:
        phase_measurements (list): List of phase values [phi_0, phi_1, phi_2] in degrees
        f (float): Frequency in Hz (e.g., 2.4e9 for 2.4 GHz)
        d_factor (float): Element spacing as a fraction of wavelength (default 0.5λ)
    
    Returns:
        float: Estimated Angle of Arrival (AoA) in degrees
    """
    c = 3e8  # Speed of light in m/s
    lambda_ = c / f  # Wavelength in meters
    d = d_factor * lambda_  # Element spacing

    # Convert phase to radians
    phi_0, phi_1, phi_2 = np.radians(phase_measurements)

    # Compute phase difference (ensure wrapping correction)
    delta_phi = np.unwrap([phi_0, phi_1, phi_2])[1] - phi_0  # Between antenna 0 and 1

    # Estimate AoA
    theta_rad = np.arcsin((lambda_ * delta_phi) / (2 * np.pi * d))
    theta_deg = np.degrees(theta_rad)

    return theta_deg

# Example usage
phase_meas = [0, 90, 180]  # Example phase in degrees
f = 2.4e9  # 2.4 GHz frequency

aoa = estimate_aoa(phase_meas, f)
print(f"Estimated AoA: {aoa:.2f} degrees")

Estimated AoA: 30.00 degrees


## Beamformer in Reality

This section will include theory on the possibility for implementing beamforming on SDR.

Main inspiration taken from [Jon Kraft GitHub](https://github.com/jonkraft/Pluto_Beamformer.git)

### Pseudocode 

**List all pseudocode**



#### Plot FFT

1. Basic Setup
2. Sent Standard sine wave
3. function for reading raw data in IQ to FFT (dbfs)
4. Get data from both channel
5. Create empty array for peak sum
6. loop through delay phases of -180 to 180 with 2 increment
      1. In the loop, change the phase in one channel according to the loop and add calibration
      2. Sum both channel in fft (delay Rx[1] and Rx[0])

#### Plot Peaks

1. Set sample rate, gain, freq center
2. Add phase calibration because cable length and any other
3. Set distance between Rx antennas, calculate distance by half of the wavelength. To know the distance in mm, multiply by 1000
4. Function to calculate theta with phase in degree
      1. arcsin_arg = phase in radians * 3E8 / (2 * pi * freq_center * d_Rx)
      2. arcsin_arg = max(min(1,arcsin_arg),-1)
      3. calculate theta = np.rad2deg(np.arcsin(arcsin_arg))
      4. Return calculate theta
5. Function for reading raw data in IQ to FFT (dbfs)
6. Create empty array for peak sum
7. loop through delay_phases of -180 to 180 with 2 increment
      1. In the loop, change the phase in one channel according to the loop and add calibration
      2. Sum both channel in fft (delay Rx[1] and Rx[0])
      3. Get the peak_sum
8. peak_dbfs is maximum of the peak_sum
9. Get index of peak_delay by using np.where at which the peak_sum equal to peak_dbfs
10. peak_delay is delay_phases with index of peak_delay_index